In [1]:
%pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import transformers

print(transformers.__version__)

4.41.1


In [4]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_from_scratch_notebook", framework="pytorch")

## Preparing the dataset

In [5]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,text
0,= = = Alteration to the law = = = \n
1,"Pfaster , who was behind the exhumations , proves Mulder 's prediction correct when he brings a prostitute to his apartment . When the prostitute discovers a collection of funerary wreaths in Pfaster 's bedroom , he kills her and removes her fingers . Later , Pfaster — having been hired as a frozen food delivery man through charming the female interviewer — delivers to a low @-@ security house of a woman with teenage daughters . He requests the bathroom to wash his hands , and whilst there steals some discarded hair from a brush he found in the trashcan . Pfaster attends a night class at a community college , where a female classmate defends herself after he makes threatening advances . He is arrested and is placed in a jail cell across from a suspect being interrogated for Pfaster 's crimes by Mulder , Scully , and Bocks . Pfaster shows interest in Scully , and learns her name from the interrogated suspect . Pfaster is later released as his charges were dropped . \n"
2,
3,
4,= = = Climate = = = \n
5,"Detachment from riches is the goal of the tenth commandment and the first Beatitude ( "" blessed are the poor in spirit "" ) because , according to the Catechism , this precept is necessary for entrance into the Kingdom of heaven . Covetousness is prohibited by the tenth commandment because it is considered to be the first step toward commission of theft , robbery and fraud ; these lead to violence and injustice . The Church defines covetousness as a "" disordered desire "" that can take different forms : \n"
6,
7,
8,"Now look at the second bit of the quote from Chicago , chastely separated from the preceding sentence by a demure three @-@ point ellipsis . As my associate Kate Levin has discovered , this passage ( "" To cite a source from a secondary source ... "" ) occurs on page 727 , which is no less than 590 pages later than the material before the ellipsis , in a section titled "" Citations Taken from Secondary Sources . "" Here 's the full quote , with what Dershowitz left out set in bold : "" ' Quoted in ' . To cite a source from a secondary source ( "" quoted in "" ) is generally to be discouraged , since authors are expected to have examined the works they cite . If an original source is unavailable , however , both the original and the secondary source must be listed . "" \n"
9,= = = Gender Advertisements = = = \n


## Causal Language modeling

In [10]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)



In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [14]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [13]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [16]:
tokenized_datasets["train"][1]

{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [15]:
# block_size = tokenizer.model_max_length
block_size = 128

In [17]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [18]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [19]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

In [20]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
from transformers import Trainer, TrainingArguments

In [22]:
%pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [22]:
%pip install transformers[torch]

In [24]:
%pip install accelerate>=0.21.0

In [23]:
%pip install --upgrade transformers torch


In [24]:
training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,6.552100,6.469504
2,6.188100,6.198531


Epoch,Training Loss,Validation Loss
1,6.552100,6.469504
2,6.188100,6.198531
3,6.009200,6.112238


TrainOutput(global_step=6747, training_loss=6.386851838664406, metrics={'train_runtime': 2458.9166, 'train_samples_per_second': 21.95, 'train_steps_per_second': 2.744, 'total_flos': 3525678710784000.0, 'train_loss': 6.386851838664406, 'epoch': 3.0})

In [36]:
import math
import torch

# Evaluate the model and calculate perplexity
try:
    # Evaluate the model and store the results
    eval_results = trainer.evaluate()

    # Calculate and print the perplexity
    perplexity = math.exp(eval_results['eval_loss'])
    print(f"Perplexity: {perplexity:.2f}")

    # Print all evaluation results
    print("Evaluation results:")
    for key, value in eval_results.items():
        print(f"{key}: {value}")

except KeyError as e:
    print(f"Key error: {e} not found in evaluation results.")
except Exception as e:
    print(f"An error occurred: {e}")

prompt = "The quick brown fox"

# Ensure input_ids is on the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Generate text
output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Perplexity: 451.35
Evaluation results:
eval_loss: 6.112237930297852
eval_runtime: 22.4168
eval_samples_per_second: 86.274
eval_steps_per_second: 10.795
epoch: 3.0
The quick brown fox, and the city of the city's death. The first time, the first time was the first time of the first time. The first time, the first time was the first time in the first time, and the


In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

events.out.tfevents.1716111184.efd1fde58bfa.2159.0:   0%|          | 0.00/8.95k [00:00<?, ?B/s]

events.out.tfevents.1716114039.efd1fde58bfa.2159.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NovaBot/gpt2-wikitext2/commit/7a0877385ee0945a19e5890b8501747342549f13', commit_message='End of training', commit_description='', oid='7a0877385ee0945a19e5890b8501747342549f13', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
# Define the path to save the model
model_save_path = '/content/drive/My Drive/gpt2-wikitext2-model.pth'

# Save the model
torch.save(model.state_dict(), model_save_path)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary libraries
!pip install transformers datasets

# Import necessary libraries
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import torch

In [45]:
# Define the path to the saved model in Google Drive
model_save_path = '/content/drive/My Drive/gpt2-wikitext2-model.pth'
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

# Load the model configuration and initialize the model
model_checkpoint = "gpt2"
config = AutoConfig.from_pretrained(model_checkpoint)
loaded_model = AutoModelForCausalLM.from_config(config)

# Load the saved model state dictionary
loaded_model.load_state_dict(torch.load(model_save_path))

# Set the model to evaluation mode
loaded_model.eval()

# Function to generate text
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = loaded_model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example prompt for text generation
prompt = "The quick brown fox"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: The quick brown fox, and the city of the city's death. The first time, the first time was the first time of the first time. The first time, the first time was the first time in the first time, and the
